<h1><font size="8">Simple Linear Regression</font></h1>

Created by: M. Ben-Azzouz (moez.ben-azzouz@sinclair.edu)

# Part I - Background

<p>In this notebook, we will explore an automobile dataset containing fuel consuption and carbon dioxide emission data of various cars. We will use Pandas and Matplotlib (Python libraries) to explore the data, visualize it, and train a simple linear regression model that can be used to predict unknown values. Part I will serve as the tutorial and background for this lab, so the student is expected to run each cell and examine the code and output to learn from this portion. Part II is the graded portion where the student is expected to write code and complete exercises.</p>

## Importing Relevant Libraries and Packages

In [ ]:
import matplotlib.pyplot as plt     # Load matplotlib library for visualizing data
import pandas as pd                 # Load pandas library for manipulating data
import numpy as np                  # Load numpy library for numerical (specifically matrix/vector and linear algebra )
from sklearn import linear_model    # Load the linear_model class from the scikit-learn machine learning library
from sklearn.metrics import r2_score  # load the R2-score metric from the scikit-learn machine learning library
%matplotlib inline

## Understanding the Data

### `FuelConsumption.csv`:
<p>We have downloaded a fuel consumption dataset, **`FuelConsumption.csv`**, which contains model-specific fuel consumption ratings and estimated carbon dioxide emissions for new light-duty vehicles for retail sale in Canada.</p>

- **MODELYEAR** e.g. 2014
- **MAKE** e.g. Acura
- **MODEL** e.g. ILX
- **VEHICLE CLASS** e.g. SUV
- **ENGINE SIZE** e.g. 4.7
- **CYLINDERS** e.g 6
- **TRANSMISSION** e.g. A6
- **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
- **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0

## Loading the Data

<p>We will first load the data into a Pandas dataframe. Prior to doing this, make sure that the file FuelConsumption.csv is in the same directory as this notebook.</p>

In [ ]:
df = pd.read_csv("FuelConsumption.csv")
df.head()                                 # Let's take a look at a sample of the first five observations

## Exploring the Data

<p>Let's first use the describe() method to get a summary of the data. This command provides summary statistics (mean, standard deviation, minimum, maximum, quartiles, and count) of numeric features (variables). If a feature is categorical (nonnumeric), it is not included in this summary.</p>

In [ ]:
df.describe()

Let's take a closer look at certain features

In [ ]:
# We will create a new dataframe that contains only the data of the four specified features.
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]

# Let's look at the first ten observations (samples) from this new dataframe 
cdf.head(10)

Let's visualize the distributions of these four features using histograms

In [ ]:
viz_data = cdf[['CYLINDERS','ENGINESIZE','CO2EMISSIONS','FUELCONSUMPTION_COMB']]
viz_data.hist()
plt.show()

<p>We can make the following observations from the historgrams:</p>
<ul>
    <li>The distribution of CO2 emissions is roughly normal (bell-shaped)</li>
<li>The engine size distribution is skewed to the right, which suggest that most of the cars sold have a smaller engine size.</li>
<li>The combined fuel consumption (L/km) distribution indicates that most cars consume around 10 liters per kilometer</li> 
<li>The distribution of cylinders is skewed to the right and has clear gaps and spikes at certain values. This makes perfect sense since there are less vehicle with a number of cylinders above 8 and the number of cylinders is an integer.</li></ul>

<p>Let's use a scatter plot to visualize the relationship between two features. For example, the combined (city and highway) fuel consumption (L/km) versus CO2 emissions.</p>

In [ ]:
# Create a scatter plot with fuel consumption on the x-axis and emissions on y-axis and the dot colored in blue
plt.scatter(cdf.FUELCONSUMPTION_COMB, cdf.CO2EMISSIONS,  color='blue')

# Assign an appropriate label to each axis
plt.xlabel("Combined Fuel Consumption (L/km)")
plt.ylabel("CO2 Emissions")

# Invoke the show() method to display the plot
plt.show()

Let's do the same with CO2 emissions versus engine size

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

<p>The scatter plot above suggests a strong positive linear correlation between engine size and CO2 emissions, which is a logical relationship. We can take advantage of this observation and try to fit a linear regression model (best fit line) to the data. This model can then be used to predict CO2 emissions for engine size values (not necessarily in our dataset).</p> 

<p>We will complete the following steps to achieve this:</p>
<ol>
    <li>Split the data into a training set and a testing set</li>
    <li>Train/Fit a linear regression model to the training set</li>
    <li>Apply the model to the test data and make predictions</li>
    <li>Evaluate the model using various metrics</li>
</ol>
<p>We will complete step 1 for you and you will be asked to complete steps 2-4 in Part II. I know that sounds like a daunting task. However, we will first provide you with a full discussion of the theory behind fitting a linear regression model and walk you through an example where we apply the four steps to some made up data.</p>

<h2 id="simple_regression">Simple Regression Model</h2>
<h3>Univariate Case (N = 1)</h3>
<p>Linear Regression fits a linear model with coefficients $\theta = (\theta_0, ..., \theta_n)$ to minimize the 'residual sum of squares' between the independent x in the dataset, and the dependent y by a linear approximation.</p>

<p>We will look at simple linear regression where both x and y are one-dimensional (scalars). We seek a linear model with equation $y = \theta_1x + \theta_0$ for example where x and y are univariate. More specifically, we want to know the values of the parameters $\theta_1$ (called the slope or gradient) and $\theta_0$ (called the y-intercept or bias term) such that the line is the best fit for the data (as seen in the figure below)</p>

![Simple Linear Regression](https://i.stack.imgur.com/zoYKG.png)

<p><em>How do we determine the parameters for the best fit line?</em></p>

<p>We want to choose the line that is closest to all data points, so we pick the parameters such that the sum of the errors or vertical distances (indicated in green in the figure) between data points and the line is minimized. In fact, instead of trying to minimize the sum of the errors, we typically opt for minimizing the sum of the squares of the errors. This has two benefits: it removes the negative sign when it occurs in the difference and amplifies the impact of larger errors on the overall sum in relation to smaller ones. This sum is often referred as residual sum or squares or sum of squared errors (SSE) and the method of minimizing it is often called Ordinary Least Squares (OLS).</p> 

<p>So, if we let $\hat{y}$ refer to the y-value predicted by our best fit line and $y$ refer to the true y-value for a given $x$, then the square of the error is given by $(\hat{y} - y)^2$. We sum this quantity across all data points (ordered pairs of x and y values) in our dataset. If we let D be the dimension of the data (number of data points), then the SSE (Sum of Squared Errors), which incidentally is also the square of the length (norm) of the difference vector $\hat{y} - y$ be is given by the following</p>
<p>
$$
\begin{eqnarray} 
& SSE = \sum^D_{j=1}{(\hat{y} - y)^2} = {\lVert \hat{y} - y\rVert^2} \\
&= {\lVert \boldsymbol \theta^T\boldsymbol{X} - y \rVert^2} \\
&= (\boldsymbol X\boldsymbol {\theta} - y)^T(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y),
\end{eqnarray}
$$</p>
<p>where $\hat{y} = \theta_1\boldsymbol x + \theta_0$, $\boldsymbol x$ is a 1xD vector 
(it's actually an NxD matrix but N = 1 in this case).</p> 

<p>$\hat{y}$ has been further re-written as $\theta^TX$, i.e., the product of the vector $[\theta_0\>\>\theta_1]^T$ and the (N+1)xD (2xD in this case) matrix $\boldsymbol X = [1\>\>\boldsymbol x]^T$ ($\boldsymbol x$ has been augmented by $x_0 = 1$ to form $X$ to account for the coefficient of $\theta_0$. $\boldsymbol{\theta}$ is an (N+1)x1 (2x1 in this case) vector.</p>

<p>To minimize the SSE, we resort to calculus. We find the gradient of the SSE with respect to $\boldsymbol\theta$ (in this case we take the partial derivatives with respect to $\theta_0$ and $\theta_1$), set it equal to zero, and solve for $\boldsymbol\theta$ (or in this case $\theta_0$ and $\theta_1$). This process will be demonstrated in the general case below.</p>

<p>Note: besides SSE (Sum of Squared Errors), there are two other common expressions: MSE (Mean Squared Errors) and RMSE (Root Mean Squared Errors). MSE is simply the SSE averaged out over the data, RMSE is the square root of the MSE. The good news is that when it comes to finding the minimum, all three expressions have the same global minimum (square rooting a function or dividing it by a constant does not affect the location of its minimum). In practice it is easier to minimize the MSE or SSE.</p> 

<h3>General Case (Multivariate Case N>1)</h3>
<p>Consider the case where we have a linear model for predicting housing prices. We are predicting the housing prices based on features in the 
housing dataset. If we denote the features as $\boldsymbol x_1, \dotsc, \boldsymbol x_N$ and collect them into an NxD feature matrix, and the prices of the houses as $y$. Assuming that we have 
a prediction model in this way such that $\hat{y}_i =  f(\boldsymbol {x}_i) = \boldsymbol \theta^T\boldsymbol {x}_i$.</p>



<p>If we collect the data into a $(N+1,D)$ data matrix $X$ (NxD feature matrix augmented by a $x_0^T$ row of ones), we can write down our model like this:

$$
\begin{bmatrix} 
\boldsymbol{x}_1^T \\
\vdots \\ 
\boldsymbol{x}_N^T 
\end{bmatrix} \boldsymbol{\theta} = \begin{bmatrix} 
y_1 \\
\vdots \\ 
y_D 
\end{bmatrix},
$$

i.e.,

$$
\boldsymbol X\boldsymbol{\theta} = \boldsymbol{y}.
$$

Note that the data points are the *rows* of the data matrix, i.e., every row is a dimension (D) of the data. 

Our goal is to find the best $\boldsymbol\theta$ such that we minimize the following objective (least squares).

$$
\begin{eqnarray} 
& \sum^N_{i=0}{\lVert \hat{y_i} - y_i \rVert^2} \\
&= \sum^N_{i=0}{\lVert \boldsymbol \theta^T\boldsymbol{x}_i - y_i \rVert^2} \\
&= (\boldsymbol X\boldsymbol {\theta} - \boldsymbol y)^T(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y).
\end{eqnarray}
$$

If we set the gradient of the above objective to $\boldsymbol  0$, we have
$$
\begin{eqnarray} 
\nabla_\theta(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y)^T(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y) &=& \boldsymbol 0 \\
\nabla_\theta(\boldsymbol {\theta}^T\boldsymbol X^T - \boldsymbol y^T)(\boldsymbol X\boldsymbol {\theta} - \boldsymbol y) &=& \boldsymbol 0 \\
\nabla_\theta(\boldsymbol {\theta}^T\boldsymbol X^T\boldsymbol X\boldsymbol {\theta} - \boldsymbol y^T\boldsymbol X\boldsymbol \theta - \boldsymbol \theta^T\boldsymbol X^T\boldsymbol y + \boldsymbol y^T\boldsymbol y ) &=& \boldsymbol 0 \\
2\boldsymbol X^T\boldsymbol X\theta - 2\boldsymbol X^T\boldsymbol y &=& \boldsymbol 0 \\
\boldsymbol X^T\boldsymbol X\boldsymbol \theta        &=& \boldsymbol X^T\boldsymbol y.
\end{eqnarray}
$$
</p>
<p>The solution (we'll call it $\hat{\theta}$) that gives zero gradient solves (which we call the maximum likelihood estimator) the following equation:

$$\boldsymbol X^T\boldsymbol X\boldsymbol \theta = \boldsymbol X^T\boldsymbol y.$$

_This equation (or its solution below) is called the Normal Equation. Incidentally, this is exactly the same equation we would arrive at when finding the projection matrix_.

This means that if we solve for $\boldsymbol X^T\boldsymbol X\boldsymbol \theta = \boldsymbol X^T\boldsymbol y$, we would find the best $\boldsymbol \theta$: 

$$\hat{\theta} = (\boldsymbol X^T\boldsymbol X)^{-1}\boldsymbol X^T\boldsymbol y$$, i.e. the $\boldsymbol \theta$ which minimizes our objective.</p>

<h3>Interesting Connection</h3>

<p>The solution above, which we reached using <i>calculus</i>, lies at the intersection of various areas of mathematics. We can reach it by <i>geometry</i>, with the problem boiling down to finding $\hat{\theta}$ the closest point to $\boldsymbol y$ lying in the column space of $\boldsymbol X$ (meaning we are able to write the solution as a linear combination of the columns of $\boldsymbol X$). The solution is then the orthogonal projection onto the column space. If the columns of $\boldsymbol X$ are independent (meaning $\boldsymbol X$ is invertible), the solution reduces to  $\hat{\theta} = (\boldsymbol X)^{-1}\boldsymbol y$. We can also reach it by <i>algebra</i> when solving $\boldsymbol X\theta = \boldsymbol y$ by splitting $\boldsymbol y$ into the sum of two parts $\boldsymbol p$ and $\boldsymbol e$ with $p$ in the column space of $X$ and $e$ the perpendicular part to the column space.</p> 

<p>We won't go through the derivation here, but it was worth pointing out this curious convergence of <i>algebra, calculus, and geometry</i> to reach this solution.</p>

The Normal Equation computes the inverse of $\boldsymbol X^T\boldsymbol X$ which is an (N+1)x(N+1) matrix (where N is the number of features). The _computational complexity_ of inverting this matrix is typically about $O(N^3)$ depending on the implementation. This means that the algorithm's running time increases cubically as a function of the number of features. So, if the number of features is doubled, the running time is multiplied by $2^3 = 8$. Scikit-Learn uses SVD (Singular Value Decomposition) in its LinearRegression class, this reduces the complexity to about $O(n^2)$, which is a significant benefit. So, if the number of features is doubled, the running time is quadrupled. However, both approaches get very slow if the number of features is large (e.g., 100,000). Both approaches though are linear in terms of the number of instances in the training set (they are $O(M)$, M is the number of training observations). This means they can handle a large training set efficiently, provided it can fit in memory.

If we have a large number of features or too many training observations to fit in memory, then a better approach for training a linear regression model is to use the method of gradient descent. This is a numerical approximation algorithm for finding the minimum of the objective (SSE or MSE). It sidesteps using the closed-form solution (Normal Equation) and hence avoids the numerical difficulty of computing the inverse of $\boldsymbol X^T\boldsymbol X$.

## Simple Linear Regression Example

In [ ]:
# Let's make up a dataset. We will create a 100 random x-values and construct the y-values using the line y=3x+4 distored
# by some random noise
# So, the true signal is y=3x+4 but our data points are distorted off that line using the random noise
# Our linear model will try to find the signal (or get as close as possible) by fitting a line to the data
X = 2*np.random.rand(100,1)
y = 4 + 3*X + np.random.rand(100,1) 

<h2>1. Train/Test Split</h2>

<p>The Train/Test Split step involves splitting the dataset into a training and testing sets. The two sets are mutually exclusive. We typically do a 80/20 or 70/30 split with 80% or 70% of the data used for training, respectively. The reason for this split is to evaluate our model for overfitting by checking it against out-of-sample data (data unseen during the training step).</p>

<p>Let's split our dataset into train and test sets, 80% of the entire data for training, and the 20% for testing. We will need to do this randomly. A convenient way of doing this is by first creating a mask to select random rows using __np.random.rand()__ function, then assigning the rows that satisfy the mask condition to the train set and the rest to the test set:</p>

In [ ]:
# The lines below require a bit of explaining. Let's break it down piece by piece:
# The np.random.rand(d) function creates an array of size d populated with random numbers 
# greater than or equal to 0 and less than 1. So, we are first creating such an array with length
# equal to the size of the dataframe df.
# We then create a boolean (a boolean variable takes on a value of either True or False) vector named 'msk'
# that takes each value in the random vector and checks whether it is less than 0.8
# So, msk will have the following form [True, False, True, True, ..., False]
# When we then use cdf[msk], this will only include the rows in cdf where the corresponding value in msk is True
# The proportion of True values in the mask vector is 80% of the size of the vector

msk = np.random.rand(len(X)) < 0.8
train__X = X[msk]
test__X = X[~msk]
train__y = y[msk]
test__y = y[~msk]

print(train__X.shape)       # To confirm that our split occurred as planned let's print the sizes of the two sets
print(test__X.shape)
print(100*train__X.shape[0]/(train__X.shape[0] + test__X.shape[0])) # Let's check the percentage of allocated training rows

<h2>2. Train/Fit the Model</h2>

<p>We will fit a linear regression model to our data in two ways:</p>
<ul>
    <li>using the closed-form solution to the least squares problem, i.e., the normal equation, and</li>
    <li>by invoking functions from the Scikit-Learn machine learning algorithms library.</li>
</ul>
<p>It is important to see both approaches so we are not limited by the black-box approach of doing machine learning, where we just memorize which algorithm to use when and what syntax to type to invoke it. That is a decent way to start learning machine learning. However, to become a true expert, it is crucial to understand what goes on inside the box.</p>
<p>That is exactly where the mathematical foundations from linear algebra, analytic geometry, probability and statistics, and optimization can help us comprehend and navigate through the innerworkings of each algorithm. Once we fully grasp how a specific algorithm works, we learn about its strengths and weaknesses and this can empower us to modify it or create our own improved version from scratch to suit our case study.</p>
<p>Another aspect that is crucial in practice is the numerical complexity in implementing each algorithm. Although that's a topic beyond the score of this course, we will discuss it briefly in the context of simple linear regression.</p>

<h3>a. Fit the Model Using the Normal Equation</h3>

In [ ]:
# Augment the X vector by a vector of ones to account for the coefficients of the bias terms theta_0
X_b = np.c_[np.ones((len(train__X),1)),train__X]    # add X0 = 1 to each instance

# Use the least squares solution, i.e., the Normal Equation, to find the parameters that minimize the SSE
theta_best = np.linalg.inv(X_b.T@X_b)@X_b.T@train__y

# Print the parameters (intercept and slope)
print('Coefficient(s): ', theta_best[1])
print('Intercept: ', theta_best[0])

# Use the model to predict the y-values (labels) of the training inputs
y_best = X_b@theta_best

# Visualize the data and our linear model (best fit line)
plt.scatter(train__X, train__y, c='b')
plt.plot(train__X, y_best, '-g')
plt.xlabel('X (train)')
plt.ylabel('y (train)')

<h3>b. Fit the Model Using Scikit-Learn</h3>

In [ ]:
# Create a linear regression object from the sklearn library (the relevant package was already imported)
regr_model = linear_model.LinearRegression()

# Fit the model to our training data
regr_model.fit(train__X, train__y)
y_best_sklearn = regr_model.coef_*train__X + regr_model.intercept_

# Print the parameters (intercept and slope)
print('Coefficient(s): ', regr_model.coef_)
print('Intercept: ', regr_model.intercept_)

# Visualize the training data and our linear model (best fit line)
plt.scatter(train__X, train__y, color='b')
plt.plot(train__X, y_best_sklearn, '-r')
plt.xlabel('X (train)')
plt.ylabel('y (train)')

<h3>Computational Complexity</h3>

The Normal Equation computes the inverse of $\boldsymbol X^T\boldsymbol X$ which is an (N+1)x(N+1) matrix (where N is the number of features). The _computational complexity_ of inverting this matrix is typically about $O(N^3)$ depending on the implementation. This means that the algorithm's running time increases cubically as a function of the number of features. So, if the number of features is doubled, the running time is multiplied by $2^3 = 8$. Scikit-Learn uses SVD (Singular Value Decomposition) in its LinearRegression class, this reduces the complexity to about $O(N^2)$, which is a significant benefit. So, if the number of features is doubled, the running time is quadrupled. However, both approaches get very slow if the number of features is large (e.g., 100,000). Both approaches though are linear in terms of the number of instances in the training set (they are $O(M)$, M is the number of training observations). This means they can handle a large training set efficiently, provided it can fit in memory.

If we have a large number of features or too many training observations to fit in memory, then a better approach for training a linear regression model is to use the method of gradient descent. This is a numerical approximation algorithm for finding the minimum of the objective (SSE or MSE). It sidesteps using the closed-form solution (Normal Equation) and hence avoids the numerical difficulty of computing the inverse of $\boldsymbol X^T\boldsymbol X$.

<h2>3. Test the Model</h2>
<h3>a. Test the Model from the Normal Equation</h3>

In [ ]:
# Augment the test set with a column of ones
test__X_b = np.c_[np.ones((len(test__X),1)),test__X]

# Use the model to predict the labels corresponding to the test set's inputs 
test__y_hat = test__X_b@theta_best

# Visualize the test data and our linear model (best fit line)
plt.scatter(test__X, test__y, color='b')
plt.plot(test__X, test__y_hat, '-g')
plt.xlabel('X (test)')
plt.ylabel('y (test)')

<h3>b. Test the Model from Sci-Kit Learn</h3>

In [ ]:
# Use the sklearn model to predict labels from test data
# We don't have to manually augment the test data with ones, sklearn takes care of that step for us
test__y_hat_sklearn = regr_model.predict(test__X)

# Visualize the test data and our linear model (best fit line)
plt.scatter(test__X,test__y, color='b')
plt.plot(test__X, test__y_hat_sklearn, '-r')
plt.xlabel('X (test)')
plt.ylabel('y (test)')

<h2>4. Evaluate the Model</h2>

We compare the actual values and predicted values from the test set to calculate the accuracy of a regression model. Evaluation metrics provide a key role in the development of a model, as they provide insight into areas that require improvement.

There are different model evaluation metrics. We will present and demonstrate the following three: 
<ul>
    <li> Mean Absolute Error (MAE): It is the mean of the absolute value of the errors. This is the easiest of the metrics to understand since it’s just average error.</li>
    <li> Mean Squared Error (MSE): this is the mean of squared errors. It’s more popular than MAE because the focus is shifted towards large errors. This is due to the squared term exponentially increasing larger errors in comparison to smaller ones.</li>
    <li> Root Mean Squared Error (RMSE): This is the square root of the Mean Square Error. </li>
    <li> R-squared is not error, but is a popular metric for the accuracy of your model. It represents how close the data are to the fitted regression line. The higher the R-squared, the better the model fits your data. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse).</li>
</ul>

In [ ]:
# We will compute the three evaluation metrics for each model
# Please note that we have imported the relevant libraries/packages at the top of this notebook

# Compute the three metrics for the model based on the normal equation
print("Evaluation Metrics for Normal Equation Model:")
print("-"*45)
print("Mean absolute error (MAE): %.2f" % np.mean(np.absolute(test__y_hat - test__y)))
print("Mean sum of squares (MSE): %.2f" % np.mean((test__y_hat - test__y)**2))
print("R2-score: %.2f\n" % r2_score(test__y_hat , test__y))

# Compute the three metrics for the model from sklearn
print("Evaluation Metrics for the Scikit Learn Model:")
print("-"*46)
print("Mean absolute error: %.2f" % np.mean(np.absolute(test__y_hat_sklearn - test__y)))
print("Mean sum of squares (MSE): %.2f" % np.mean((test__y_hat_sklearn - test__y)**2))
print("R2-score: %.2f" % r2_score(test__y_hat_sklearn, test__y))

# Part II - Assignment
## Write your name in a markdown cell below

## Exercise 1 (Creating a Scatter Plot and Interpreting Relationship)

a. Create a scatter plot of CO2 emissions versus cylinders.

In [ ]:
# Write your code here


b. Describe what the scatter plot seems to indicate about the relationship between the two features in a markdown cell below.

## Exercise 2 (Fitting a Linear Regression Model)
<h2>1. Train/Test Split</h2>

In [ ]:
# The lines below require a bit of explaining. Let's break it down piece by piece:
# The np.random.rand(d) function creates an array of size d populated with random numbers 
# greater than or equal to 0 and less than 1. So, we are first creating such an array with length
# equal to the size of the dataframe df.
# We then create a boolean (a boolean variable takes on a value of either True or False) vector named 'msk'
# the takes each value in the random vector and checks whether it is less than 0.8
# So, msk will has the following form [True, False, True, True, ..., False]
# When we then use cdf[msk], this will only include the rows in cdf where the corresponding value in msk is True
# The proportion of True values in the mask vector is 80% of the size of the vector

msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]
print(train.shape)       # To confirm that our split occurred as planned let's print the sizes of the two sets
print(test.shape)
print(100*train.shape[0]/(train.shape[0]+test.shape[0])) # Let's check the percentage of allocated training rows

train_x = np.asanyarray(train[['ENGINESIZE']])  # Convert the Pandas dataframe to an array
train_y = np.asanyarray(train[['CO2EMISSIONS']])

<h2>2. Train/Fit the Linear Regression Model (Two Different Ways)</h2>
<h3>a. Train/Fit the Model Using the Normal Equation</h3>

In [ ]:
X = train_x.reshape(-1,1)                                                # Convert train_x to a 2D Nx1 Numpy array
Y = train_y.reshape(-1,1)                                                # Convert train_y to a 2D Nx1 Numpy array

# Augment X by a column vector of ones (essentially adding an X0 vector of ones that is the coefficient of theta_0)
X_bias =                                                                 # <-- Complete this part

# Use the normal equation to find theta_hat the vector of parameters that minimizes the SSE
theta_hat =                                                              # <-- Complete this part
y_hat =                                                                  # <-- Complete this part

# Print the parameters (intercept and slope)
print('Coefficient(s): ', theta_hat[1])
print('Intercept: ', theta_hat[0])

# Visualize the data and our linear model (best fit line)
plt.scatter(train_x, train_y, c='b')
plt.plot(X, y_hat, '-g')
plt.xlabel('Engine Size (train)')
plt.ylabel('CO2 Emissions (train)')

<h3>b. Train/Fit the Model Using Scikit Learn</h3>

In [ ]:
# Create a linear regression object from the sklearn library
regr =                                                                 # <-- Complete this part

# Fit the model to our training data
regr.fit('complete this part')                                         # <-- Complete this part
y_hat_sklearn =                                                        # <-- Complete this part


# Print the parameters (intercept and coefficient(s)
                                                                       # <-- Complete this part 
                                                                       # <-- Complete this part
                                                                        

# Visualize the training data and our linear model (best fit line)
plt.scatter(train_x, train_y, color='b')
plt.plot(train_x, y_hat_sklearn, '-r')
plt.xlabel('Engine Size (train)')
plt.ylabel('CO2 Emissions (train)')

<h2>3. Test the Model</h2>
<h3>a. Test the Model from the Normal Equation</h3>

In [ ]:
test_x = np.asanyarray(test[['ENGINESIZE']])
test_y = np.asanyarray(test[['CO2EMISSIONS']])

# Augment the test set with a column of ones
test_x_bias =                                                               # <-- Complete this part

# Predict the labels of the test data
test_y_hat =                                                                # <-- Complete this part

# Visualize the test data and our linear model (best fit line)
plt.scatter(test_x, test_y, color='b')
plt.plot(test_x, test_y_hat, '-g')
plt.xlabel('Engine Size (test)')
plt.ylabel('CO2 Emissions (test)')

<h3>b. Test the Model from Scikit Learn</h3>

In [ ]:
# We do not need to augment test_x with a column of ones, sklearn does that step for us.
# Use the sklearn model to predict labels from test data
test_y_hat_sklearn =                                               # <-- Complete this part

# Visualize the test data and our linear model (best fit line)
plt.scatter(test_x,test_y, color='b')
plt.plot('complete this part, graph the model in red')             # <-- Complete this part
plt.xlabel('Engine Size (test)')
plt.ylabel('CO2 Emissions (test)')

<h2>4. Evaluate the model</h2>

In [ ]:
# Find the MAE, MSE, and R^2 of the two models
# The relevant package(s) have already been imported

# Compute the three metrics based on the normal equation model
print("Evaluation Metrics for Normal Equation Model:")
print("-"*45)
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_hat - test_y)))
print("Mean sum of squares (MSE): %.2f" % np.mean((test_y_hat - test_y)**2))
print("R2-score: %.2f" % r2_score(test_y_hat , test_y))

# Compute the three metrics based on the sklearn model
print("\nEvaluation Metrics for the Scikit Learn Model:")
print("-"*46)
                                                                                            # <-- Complete this part
                                                                                            # <-- Complete this part
                                                                                            # <-- Complete this part